In [1]:
import pandas as pd

In [2]:
from utils.lipht_visualization import word_frequency_barplot, topic_distribution_barplot
from utils.lipht_lda_utils import PrepareDictionary, df_lda_features, get_topics_and_probability, get_lda_topics, df_lda_preprocessing, TrainLDAModel, lda_predict_df
from utils.lipht_data import getEngine

In [3]:
engine =  getEngine('LIPHT-VM-01','Akademikernes_MSCRM_Addition')

In [4]:
query="""
SELECT *
  FROM [Akademikernes_MSCRM_Addition].[out].[LDA_Messages_persisted]
  """
df_scope = pd.read_sql(query, engine)

In [5]:
# df_scope = pd.read_pickle('data/LDA_Messages_persisted_with_language.pkl')
# df_scope.to_pickle('LDA_Messages_persisted.pkl')
# df_scope = pd.read_pickle('LDA_Messages_persisted.pkl')

In [6]:
df_scope.head()

,ThreadID,MemberID,ThreadStartDate,ThreadEndDate,ThreadSubject,FirstMessageID,FirstMessageBody,FirstMemberMessageID,FirstMemberMessageBody,FirstResponseFromAKAToMemberMessageID,...,HasMemberMessage,HasResponseFromAKAToMember,HasResponseFromMemberToAKA,FirstResponseFromAKAToMemberOwner,LastMessageOwner,ThreadOwner,ThreadOwnerType,ThreadResponsibleDepartment,ThreadResponsibleDepartmentTeam,ThreadResponsibleDepartmentTeamGroup
0,645A5871-F7B5-E811-82B4-0050569118B0,713B7CF3-7D5F-E811-B843-005056AD2D14,2018-09-11 19:18:26,2018-09-17 14:17:11,Difference i timerne i juli måned,6A5A5871-F7B5-E811-82B4-0050569118B0,Kære Thomas Hauerslev Andersen\nVi skal hver m...,DBE20A48-69BA-E811-864D-005056AD2D14,"Hej Dorte,\n \nJeg ved ærlig talt ikke lige hv...",03E41A5A-84BA-E811-864D-005056AD2D14,...,1,1,1,Dorte Ellekær,Dorte Ellekær,Dorte Ellekær,8,Forsikring,Kvalitetsteam,Kvalitetsteam
1,E7748CFB-27B6-E811-82B4-0050569118B0,FCABA65B-7705-E711-AC9F-005056AD2D14,2018-09-12 01:05:48,2018-09-13 09:12:52,Dit cv på jobnet.dk mangler oplysninger for at...,EE748CFB-27B6-E811-82B4-0050569118B0,Kære Mohamad Khaled Abdel Rahman\n \nVi skrive...,93D623B0-70B6-E811-864D-005056AD2D14,Kære Akademikernes A-kasse.\n \nJeg kan se at ...,89E6572B-35B7-E811-864D-005056AD2D14,...,1,1,1,Bettina Hedegaard Elkjær,Bettina Hedegaard Elkjær,Team Rådighed,9,Jobmatch,Team Rådighed,Team Rådighed
2,7A18B9F5-48B6-E811-82B4-0050569118B0,0D96624D-638F-E311-BF81-005056AD2D14,2018-09-12 05:01:58,2018-09-13 12:03:58,Giv oplysninger om jobsøgningen og bliv ved me...,8218B9F5-48B6-E811-82B4-0050569118B0,"Kære Lena Arndal \n \nVi ved, at du er i fuld ...",70BD297A-88B6-E811-864D-005056AD2D14,"Hej \nJeg kan se, at oplysninger fra min joblo...",126F1F17-4DB7-E811-864D-005056AD2D14,...,1,1,1,Bettina Hedegaard Elkjær,Bettina Hedegaard Elkjær,Team Rådighed,9,Jobmatch,Team Rådighed,Team Rådighed
3,E1BAC568-59B6-E811-82B4-0050569118B0,DA99065D-2E03-E711-AC9F-005056AD2D14,2018-09-12 06:59:45,2018-09-12 07:20:05,Hvornår blev du færdiguddannet?,E7BAC568-59B6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nVi har modt...,A460B937-5BB6-E811-864D-005056AD2D14,"Okay. Jeg tror, jeg har været lidt forvirret.\...",6874B044-5CB6-E811-82B4-0050569118B0,...,1,1,1,Victoria Krogdal Larsen,Victoria Krogdal Larsen,Victoria Krogdal Larsen,8,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam
4,778B1D72-5BB6-E811-82B4-0050569118B0,46841B8B-38D5-E411-B68E-0050569118B0,2018-09-12 07:14:11,2018-10-10 07:48:04,Din timeindberetning i maj og juni måned,7D8B1D72-5BB6-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nTak for dit svar ved...,C3A97562-ECBF-E811-864D-005056AD2D14,"Hej Gitte,\n \nbeklager sent svar. Jeg glemte ...",F3E66444-8DC0-E811-82B4-0050569118B0,...,1,1,1,Gitte Ryom,Gitte Ryom,Gitte Ryom,8,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam


In [7]:
departmentteam = 'Language'
initiatedby = ''
n_gram = 3
sample_size= 10000
no_above = 1#0.00008#0.00003145
no_below= 10 # filter out tokens that appear in less than 15 documents
random_state=1
lda_num_topics = 10 # The number of topics
lda_chunksize = 5000
lda_passes = 50

log = {
    'departmentteam': departmentteam,
    'initiatedby': initiatedby,
    'n_gram': n_gram,
    'no_above': no_above,
    'no_below': no_below,
    'random_state': random_state,
    'lda_num_topics': lda_num_topics,
    'lda_chunksize': lda_chunksize,
    'lda_passes': lda_passes
}

In [8]:
# df_team = df_scope[(df_scope['FirstMessageBody'].isnull()==False) & (df_scope['ThreadResponsibleDepartmentTeamGroup'].str.contains(departmentteam)==True)].copy(deep=True)
df_team = df_scope[df_scope['FirstMemberMessageBody'].isnull()==False].copy(deep=True)
log['rows'] = df_team.shape[0]

In [9]:
df_team.shape[0]

115140

In [10]:
df_team = df_team#.sample(10000)

In [11]:
df_team.fillna(" ")

,ThreadID,MemberID,ThreadStartDate,ThreadEndDate,ThreadSubject,FirstMessageID,FirstMessageBody,FirstMemberMessageID,FirstMemberMessageBody,FirstResponseFromAKAToMemberMessageID,...,HasMemberMessage,HasResponseFromAKAToMember,HasResponseFromMemberToAKA,FirstResponseFromAKAToMemberOwner,LastMessageOwner,ThreadOwner,ThreadOwnerType,ThreadResponsibleDepartment,ThreadResponsibleDepartmentTeam,ThreadResponsibleDepartmentTeamGroup
0,645A5871-F7B5-E811-82B4-0050569118B0,713B7CF3-7D5F-E811-B843-005056AD2D14,2018-09-11 19:18:26,2018-09-17 14:17:11,Difference i timerne i juli måned,6A5A5871-F7B5-E811-82B4-0050569118B0,Kære Thomas Hauerslev Andersen\nVi skal hver m...,DBE20A48-69BA-E811-864D-005056AD2D14,"Hej Dorte,\n \nJeg ved ærlig talt ikke lige hv...",03E41A5A-84BA-E811-864D-005056AD2D14,...,1,1,1,Dorte Ellekær,Dorte Ellekær,Dorte Ellekær,8,Forsikring,Kvalitetsteam,Kvalitetsteam
1,E7748CFB-27B6-E811-82B4-0050569118B0,FCABA65B-7705-E711-AC9F-005056AD2D14,2018-09-12 01:05:48,2018-09-13 09:12:52,Dit cv på jobnet.dk mangler oplysninger for at...,EE748CFB-27B6-E811-82B4-0050569118B0,Kære Mohamad Khaled Abdel Rahman\n \nVi skrive...,93D623B0-70B6-E811-864D-005056AD2D14,Kære Akademikernes A-kasse.\n \nJeg kan se at ...,89E6572B-35B7-E811-864D-005056AD2D14,...,1,1,1,Bettina Hedegaard Elkjær,Bettina Hedegaard Elkjær,Team Rådighed,9,Jobmatch,Team Rådighed,Team Rådighed
2,7A18B9F5-48B6-E811-82B4-0050569118B0,0D96624D-638F-E311-BF81-005056AD2D14,2018-09-12 05:01:58,2018-09-13 12:03:58,Giv oplysninger om jobsøgningen og bliv ved me...,8218B9F5-48B6-E811-82B4-0050569118B0,"Kære Lena Arndal \n \nVi ved, at du er i fuld ...",70BD297A-88B6-E811-864D-005056AD2D14,"Hej \nJeg kan se, at oplysninger fra min joblo...",126F1F17-4DB7-E811-864D-005056AD2D14,...,1,1,1,Bettina Hedegaard Elkjær,Bettina Hedegaard Elkjær,Team Rådighed,9,Jobmatch,Team Rådighed,Team Rådighed
3,E1BAC568-59B6-E811-82B4-0050569118B0,DA99065D-2E03-E711-AC9F-005056AD2D14,2018-09-12 06:59:45,2018-09-12 07:20:05,Hvornår blev du færdiguddannet?,E7BAC568-59B6-E811-82B4-0050569118B0,Kære Niels Peter Brandt-Henriksen\nVi har modt...,A460B937-5BB6-E811-864D-005056AD2D14,"Okay. Jeg tror, jeg har været lidt forvirret.\...",6874B044-5CB6-E811-82B4-0050569118B0,...,1,1,1,Victoria Krogdal Larsen,Victoria Krogdal Larsen,Victoria Krogdal Larsen,8,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam
4,778B1D72-5BB6-E811-82B4-0050569118B0,46841B8B-38D5-E411-B68E-0050569118B0,2018-09-12 07:14:11,2018-10-10 07:48:04,Din timeindberetning i maj og juni måned,7D8B1D72-5BB6-E811-82B4-0050569118B0,Kære Cecilie Bech Fejfer\nTak for dit svar ved...,C3A97562-ECBF-E811-864D-005056AD2D14,"Hej Gitte,\n \nbeklager sent svar. Jeg glemte ...",F3E66444-8DC0-E811-82B4-0050569118B0,...,1,1,1,Gitte Ryom,Gitte Ryom,Gitte Ryom,8,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam
5,E15E85CB-5EB6-E811-82B4-0050569118B0,C1B3B38C-3807-E411-BCEF-005056AD2D14,2018-09-12 07:38:14,2018-09-12 11:13:28,"Husk, at du skal til Velkomstmøde, Aarhus",EF5E85CB-5EB6-E811-82B4-0050569118B0,"Kære Anne Nymand-Grarup\n \nHusk, at du skal t...",D6B21804-60B6-E811-864D-005056AD2D14,"Hej\nDa jeg har fået arbejde pr. 10.9.2018, ko...",21FDCFDC-7CB6-E811-82B4-0050569118B0,...,1,1,1,Charlotte Mathiesen,Charlotte Mathiesen,Charlotte Mathiesen,8,Jobmatch,Logistik,Logistik
6,7FF7359C-61B6-E811-82B4-0050569118B0,132BAC84-838F-E311-BF81-005056AD2D14,2018-09-12 07:58:21,2018-09-13 10:17:20,bekræft sidste barselsdag,86F7359C-61B6-E811-82B4-0050569118B0,Kære Pauline Lindskov Sachs\nSom aftalt skal d...,25703119-B8B6-E811-864D-005056AD2D14,Kære Lis Ulrichsen\nJeg bekræfter hermed at mi...,87D6CC2E-3EB7-E811-82B4-0050569118B0,...,1,1,1,Lis Ulrichsen,Lis Ulrichsen,Lis Ulrichsen,8,Forsikring,Udbetalingsteam 1 - CPR 01 til 10,Udbetalingsteam
7,2C6BDAD5-62B6-E811-82B4-0050569118B0,B1D9E5D8-DBA3-E511-B263-005056AD2D14,2018-09-12 08:07:05,2018-09-13 08:37:27,Mangler oplysninger til behandling af ledighed...,326BDAD5-62B6-E811-82B4-0050569118B0,Kære 

In [12]:
df_team.isnull().sum()

ThreadID                                       0
MemberID                                       0
ThreadStartDate                                0
ThreadEndDate                                  0
ThreadSubject                                  1
FirstMessageID                                 0
FirstMessageBody                               0
FirstMemberMessageID                           0
FirstMemberMessageBody                         0
FirstResponseFromAKAToMemberMessageID          0
FirstResponseFromAKAToMemberMessageBody        0
FirstResponseFromMemberToAKAMessageID      56580
FirstResponseFromMemberToAKAMessageBody    56580
LastMessageID                                  0
LastMessageBody                                0
ThreadInitiatedBy                              0
ThreadClass                                    0
HasMemberMessage                               0
HasResponseFromAKAToMember                     0
HasResponseFromMemberToAKA                     0
FirstResponseFromAKA

In [13]:
%%time
df_lda_preprocessing(df_team, 'FirstMemberMessageBody')

C:\ProgramData\Anaconda3\envs\lda\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\envs\lda\lib\site-packages\bs4\__init__.py:336: UserWarning: "https://www.stepstone.dk/jobannonce/304958/koordinator-acer-eller-professionsbachelor-til-nationalpark-kongernes-nordsjaellands-sekretariat?lang=en
 " looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\ProgramData\Anaconda3\envs\lda\lib\site-packages\bs4\__init__.py:336: UserWarning: "https://www.eal.dk/uddannelser/it-teknolog/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and 

Wall time: 10min 3s


In [14]:
df_team.columns

Index(['ThreadID', 'MemberID', 'ThreadStartDate', 'ThreadEndDate',
       'ThreadSubject', 'FirstMessageID', 'FirstMessageBody',
       'FirstMemberMessageID', 'FirstMemberMessageBody',
       'FirstResponseFromAKAToMemberMessageID',
       'FirstResponseFromAKAToMemberMessageBody',
       'FirstResponseFromMemberToAKAMessageID',
       'FirstResponseFromMemberToAKAMessageBody', 'LastMessageID',
       'LastMessageBody', 'ThreadInitiatedBy', 'ThreadClass',
       'HasMemberMessage', 'HasResponseFromAKAToMember',
       'HasResponseFromMemberToAKA', 'FirstResponseFromAKAToMemberOwner',
       'LastMessageOwner', 'ThreadOwner', 'ThreadOwnerType',
       'ThreadResponsibleDepartment', 'ThreadResponsibleDepartmentTeam',
       'ThreadResponsibleDepartmentTeamGroup', 'text', 'tokenized_text',
       'ngrams', 'stopwords_removed'],
      dtype='object')

In [16]:
# line_sample = 2
# df_team['FirstMemberMessageBody'][line_sample]

In [17]:
# df_team['text'][line_sample]

In [18]:
df_team.columns

Index(['ThreadID', 'ThreadStartDate', 'ThreadEndDate', 'ThreadSubject',
       'FirstMessageID', 'FirstMessageBody', 'FirstMemberMessageID',
       'FirstMemberMessageBody', 'FirstResponseFromAKAToMemberMessageID',
       'FirstResponseFromAKAToMemberMessageBody',
       'FirstResponseFromMemberToAKAMessageID',
       'FirstResponseFromMemberToAKAMessageBody', 'LastMessageID',
       'LastMessageBody', 'ThreadInitiatedBy', 'ThreadClass',
       'HasMemberMessage', 'HasResponseFromAKAToMember',
       'HasResponseFromMemberToAKA', 'ThreadResponsibleDepartment',
       'ThreadResponsibleDepartmentTeam',
       'ThreadResponsibleDepartmentTeamGroup', 'text', 'text_CharCount',
       'text_LessThan5000', 'text_WordCount', 'text_Questionmarks',
       'text_1Question', 'text_Exclamationmarks', 'tokenized_text', 'bigrams',
       'trigrams', 'ngrams', 'stopwords_removed'],
      dtype='object')

In [15]:
%%time
dictionary, corpus = PrepareDictionary(df_team, 'tokenized_text', no_above, no_below, log)

C:\ProgramData\Anaconda3\envs\lda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 14.9 s


In [16]:
df_team['tokenized_text'].head(10)

0    [jeg, ved, ærlig, talt, ikke, lige, hvad, der,...
1    [jeg, kan, se, at, i, meddelsen, der, skriver,...
2    [jeg, kan, se, at, oplysning, fra, min, joblog...
3    [okay, jeg, tror, jeg, har, været, lidt, forvi...
4    [beklager, sent, svar, jeg, glemme, det, helt,...
5    [da, jeg_har_modtaget, arbejde, pr, kommer, je...
6    [bekræfte, hermed, at, min, sidste, dag, med, ...
7    [jeg, er, en, del, af, et, byggge, projekt, hv...
8    [jeg, har, nu, tjekkert, det, igennem, og, kon...
9    [ikke, tage, telefon, i, dag, da, jeg, arbejde...
Name: tokenized_text, dtype: object

In [17]:
log

{'departmentteam': 'Language',
 'initiatedby': '',
 'n_gram': 3,
 'no_above': 1,
 'no_below': 10,
 'random_state': 1,
 'lda_num_topics': 10,
 'lda_chunksize': 5000,
 'lda_passes': 50,
 'rows': 115140,
 'words_preprocess': 101128,
 'words_postprocess': 12646}

In [18]:
len([d for d in dictionary.values()])

12646

In [ ]:
# 0.000031433665000 - øvre - words_postprocess = 1000000
# 0.000031433665000 - nedre - words_postprocess = 0

In [20]:
%%time
LDAmodel_scope = TrainLDAModel(corpus, dictionary, lda_num_topics, 4, lda_chunksize, lda_passes, random_state, log)

Wall time: 18min 41s


In [21]:
import os
cur_dir = os.getcwd()
# LDAmodel_scope.save(os.path.join(cur_dir,'language_model','{0}_model'.format(log['departmentteam'])))

In [31]:
from utils.lipht_lda_utils import df_lda_features
df_lda_features(LDAmodel_scope, df_team)

In [100]:
t = LDAmodel_scope.print_topic(9, topn=5)

In [101]:
print(t)

0.320*"!" + 0.047*"nummer" + 0.042*"jag" + 0.038*"sverige" + 0.031*"cpr"


In [102]:
# t = t.split("+")

In [105]:
# def topics_to_rows():
s = []
for i in t.split("+"):
    i0, i1 = i.split('*')
    s.append((float(i0.strip()),i1.split('"')[1]))

In [119]:
s

[(0.32, '!'),
 (0.047, 'nummer'),
 (0.042, 'jag'),
 (0.038, 'sverige'),
 (0.031, 'cpr')]

{0.32: '!', 0.047: 'nummer', 0.042: 'jag', 0.038: 'sverige', 0.031: 'cpr'}

In [23]:
#topic_distribution_barplot(LDAmodel_scope, df_team)

In [240]:
def get_topics_and_probability(df, lda_model, n_topics, n_topwords):
    import numpy
    import pandas as pd

    mean_df_lda_features = df['lda_features'].mean()

    def get_topic_and_prob(lda_model, topic_id, nr_top_words=7):
        id_tuples = lda_model.print_topic(topic_id, topn=nr_top_words)
        id_list = []
        for i in id_tuples.split("+"):
            i0, i1 = i.split('*')
            id_list.append((float(i0.strip()),i1.split('"')[1]))
        return id_list

    topics_dist = pd.DataFrame()

    for x in sorted(numpy.argsort(mean_df_lda_features)[-n_topics:]):
        top_words = get_topic_and_prob(lda_model, x, n_topwords) #get_topic_top_words(LDAmodel, x)
        # print("Topic: {0}\tProb: {1:.3f}, Words: {2}.".format(x, RequestTopicDistribution.item(x), top_words)) #(x, ", ".join(top_words)))
        row = {
            'Topic': x,
            'Topic_Distribution': mean_df_lda_features.item(x),
            'TopWords': top_words,
        }
        topics_dist = topics_dist.append(row, ignore_index=True)
    
    topics_dist.index.names = ['topic_index']
    topics_dist['topic_index'] = topics_dist.index

    df_topwords = topics_dist.apply(lambda x: pd.Series(x['TopWords']),axis=1).stack()#.reset_index(level=1, drop=True)
    df_topwords.name = 'prob_words'
    df_topwords = pd.DataFrame(df_topwords)
    df_topwords[['Probability_Word','Word']] = df_topwords['prob_words'].apply(pd.Series)
    df_topwords.index.names = ['topic_index','word_index']
    df_topwords['topic_index'] = df_topwords.index.get_level_values('topic_index')
    df_topwords['word_index'] = df_topwords.index.get_level_values('word_index')
    
    df = pd.merge(df_topwords, topics_dist, on='topic_index')
    df.drop(['TopWords', 'topic_index','prob_words'], axis=1, inplace=True)

    return df

get_topics_and_probability(df_team, LDAmodel_scope, lda_num_topics, 15)

C:\ProgramData\Anaconda3\envs\lda\lib\site-packages\ipykernel\__main__.py:42: FutureWarning: 'topic_index' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


,Probability_Word,Word,Topic,Topic_Distribution
0,0.074,jeg,0.0,0.172593
1,0.037,at,0.0,0.172593
2,0.034,er,0.0,0.172593
3,0.031,det,0.0,0.172593
4,0.030,i,0.0,0.172593
5,0.026,kan,0.0,0.172593
6,0.026,ikke,0.0,0.172593
7,0.023,?,0.0,0.172593
8,0.022,har,0.0,0.172593
9,0.019,på,0.0,0.172593


In [107]:
def get_topics_and_probability(df, lda_model, n_topics, n_topwords):
    import numpy
    import pandas as pd

    mean_df_lda_features = df['lda_features'].mean()

    def get_topic_and_prob(lda_model, topic_id, nr_top_words=7):
        id_tuples = lda_model.print_topic(topic_id, topn=nr_top_words)
        return id_tuples

    topics_dist = pd.DataFrame()

    for x in sorted(numpy.argsort(mean_df_lda_features)[-n_topics:]):
        top_words = get_topic_and_prob(lda_model, x, n_topwords) #get_topic_top_words(LDAmodel, x)
        # print("Topic: {0}\tProb: {1:.3f}, Words: {2}.".format(x, RequestTopicDistribution.item(x), top_words)) #(x, ", ".join(top_words)))
        row = {
            'Topic': x,
            'Probability': mean_df_lda_features.item(x),
            'TopWords': top_words,
        }
        topics_dist = topics_dist.append(row, ignore_index=True)

    return topics_dist

In [109]:
topics = get_topics_and_probability(df_team, LDAmodel_scope, lda_num_topics, 15)

In [110]:
topics

,Probability,TopWords,Topic
0,0.172593,"0.074*""jeg"" + 0.037*""at"" + 0.034*""er"" + 0.031*...",0.0
1,0.022176,"0.066*""i"" + 0.039*""danmark"" + 0.034*""flytte"" +...",1.0
2,0.116706,"0.041*""jeg"" + 0.033*""i"" + 0.028*""har"" + 0.027*...",2.0
3,0.149306,"0.085*""jeg"" + 0.034*""at"" + 0.032*""er"" + 0.029*...",3.0
4,0.092798,"0.078*""jeg"" + 0.054*""at"" + 0.027*""ikke"" + 0.02...",4.0
5,0.084415,"0.093*""jeg"" + 0.048*""den"" + 0.036*""i"" + 0.033*...",5.0
6,0.087915,"0.038*""jeg"" + 0.035*""i"" + 0.034*""at"" + 0.030*""...",6.0
7,0.150695,"0.082*""jeg"" + 0.036*""i"" + 0.032*""har"" + 0.027*...",7.0
8,0.115388,"0.071*""i"" + 0.042*""the"" + 0.038*""to"" + 0.023*""...",8.0
9,0.008007,"0.320*""!"" + 0.047*""nummer"" + 0.042*""jag"" + 0.0...",9.0


In [196]:
# topics.to_sql(name='topics_{}_{}'.format(dataset,initiatedby),con=engine , schema='input', if_exists='replace', index=False)
topics.to_sql(name='language_index' ,con=engine , schema='input', if_exists='replace', index=False)

In [34]:
get_lda_topics(df_team, LDAmodel_scope, lda_num_topics,20)

,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09
0,jeg,jeg,af,jeg,jeg,i,jeg,i,du,er
1,i,er,vedhæftet,til,har,the,at,a,vi,jeg
2,at,i,dokumentation,at,i,to,i,in,din,i
3,er,har,bibeskæftigelse,er,den,my,og,of,dig,det
4,?,min,ar,i,at,and,har,am,til,timer
5,og,at,virksomhed,en,ikke,in,det,and,tak,der
6,har,det,for,har,er,you,er,student,akasse,for
7,til,kan,lønsedler,?,fra,for,ikke,my,os,og
8,på,og,selvstændig,den,og,a,til,akasse,dit,en
9,min,til,hermed,ikke,?,have,på,member,akademikernes,på


In [35]:
import pandas as pd
df_log = pd.DataFrame(log, index=['0'])
df_log

,departmentteam,initiatedby,n_gram,no_above,no_below,random_state,lda_num_topics,lda_chunksize,lda_passes,words_preprocess,words_postprocess,train_start,train_end
0,Language,,3,1,10,1,10,5000,50,24289,3193,2018-10-16 11:45:12.118535,2018-10-16 11:52:03.247441


In [36]:
df_team.columns

Index(['ThreadID', 'ThreadStartDate', 'ThreadEndDate', 'ThreadSubject',
       'FirstMessageID', 'FirstMessageBody', 'FirstMemberMessageID',
       'FirstMemberMessageBody', 'FirstResponseFromAKAToMemberMessageID',
       'FirstResponseFromAKAToMemberMessageBody',
       'FirstResponseFromMemberToAKAMessageID',
       'FirstResponseFromMemberToAKAMessageBody', 'LastMessageID',
       'LastMessageBody', 'ThreadInitiatedBy', 'ThreadClass',
       'HasMemberMessage', 'HasResponseFromAKAToMember',
       'HasResponseFromMemberToAKA', 'ThreadResponsibleDepartment',
       'ThreadResponsibleDepartmentTeam',
       'ThreadResponsibleDepartmentTeamGroup', 'text', 'text_CharCount',
       'text_LessThan5000', 'text_WordCount', 'text_Questionmarks',
       'text_1Question', 'text_Exclamationmarks', 'tokenized_text', 'bigrams',
       'trigrams', 'ngrams', 'stopwords_removed', 'bow', 'lda_features'],
      dtype='object')

In [272]:
language = df_team[['ThreadID','ThreadInitiatedBy','FirstMemberMessageID','text','ngrams']].copy(deep=True)

In [273]:
list(language.columns)

['ThreadID', 'ThreadInitiatedBy', 'FirstMemberMessageID', 'text', 'ngrams']

In [276]:
language.head(2)

,ThreadID,ThreadInitiatedBy,FirstMemberMessageID,text,ngrams,index,prediction,pred_probability,pred_index
735443,53FEF89B-159B-E811-8F22-005056AD2D14,Member,5AFEF89B-159B-E811-8F22-005056AD2D14,,[],6,"[0.07023542, 6]",0.070235,6
1052308,C45D6ED4-DBBE-E711-B45B-005056AD2D14,AKA,36BA0E7A-DDBE-E711-B45B-005056AD2D14,,[],9,"[0.052910846, 9]",0.052911,9


In [274]:
%%time
language = lda_predict_df(language,'ngrams', LDAmodel_scope, dictionary, only_best_prediction=False)

Wall time: 14min 3s


In [177]:
values = [i for i in range(len(LDAmodel_scope.get_topics()))]
language = pd.melt(language, id_vars = ['ThreadID',
                             'ThreadInitiatedBy',
                             'FirstMemberMessageID',
                             'text',
                             'ngrams',
                             'bow'],
                   value_vars = values)
# language.rename(columns={'value':'prediction'}, inplace=True)

In [187]:
language = language[language['value'].isnull()==False].sort_values(by=['FirstMemberMessageID'])
language.rename(columns={'variable':'index','value':'prediction'}, inplace=True)
language[['pred_probability','pred_index']] = pd.DataFrame(language.prediction.values.tolist(), index= language.index)

In [277]:
language.sample(2)

,ThreadID,ThreadInitiatedBy,FirstMemberMessageID,text,ngrams,index,prediction,pred_probability,pred_index
177140,3EAA1307-54FB-E611-AC9F-005056AD2D14,Member,45AA1307-54FB-E611-AC9F-005056AD2D14,jeg har forsøgt at læse om reglerne for frivil...,"[jeg, har, forsøgt, at, læse, om, reglerne, fo...",1,"[0.13353626, 1]",0.133536,1
161610,F49938A4-0FDF-E711-8CAD-005056AD2D14,Member,FB9938A4-0FDF-E711-8CAD-005056AD2D14,jeg har forsøgt at booke en karriere samtale m...,"[jeg, har, forsøgt, at, booke, en, karriere, s...",1,"[0.01038362, 1]",0.010384,1


In [278]:
language = language[['ThreadID','ThreadInitiatedBy','FirstMemberMessageID','text','pred_probability','pred_index']]

In [279]:
language.to_sql(name='language_predictions',con=engine , schema='input', if_exists='replace', index=False)

In [280]:
language.shape[0]

375943